In [1]:
import sys
import gym
import random
import os
import numpy as np
from collections      import deque
from keras.optimizers import Adam, SGD
from keras import backend as k
import tensorflow as tf
from tensorflow import keras
from statistics import mean
from multiprocessing import Process, Queue, Lock
from skimage.transform import resize
from skimage.color import rgb2gray
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from skimage.transform import resize
from skimage.color import rgb2gray
from tensorflow.keras import layers
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Flatten

In [2]:
!pip install ray
import ray

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [3]:
class Agent():
            def __init__(self, state_size, action_size):
                self.state_size         = state_size
                self.action_size        = action_size
                self.memory             = deque(maxlen=2000)
                self.learning_rate      = 0.001
                self.gamma              = 0.95
                self.exploration_rate   = 1.0
                self.exploration_min    = 0.01
                self.exploration_decay  = 0.995
                self.model              = self._build_model()
                self.target_model       = self._build_model()
                self.discount_factor    = 0.99
                self.epsilon             = 1.
                self.optimizer          = keras.optimizers.SGD(learning_rate=0.00025, clipnorm=1.0)
            def _build_model(self):
                # Neural Net for Deep-Q learning Model\n",
                model = Sequential()
                model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu',
                                input_shape=self.state_size))
                model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
                model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
                model.add(Flatten())
                model.add(Dense(512, activation='relu')) 
                model.add(Dense(self.action_size))
                model.summary()
                return model
            def save_model(self):
                    self.model.save(self.weight_backup)
            def setModelweight(self, weights):
                self.model.set_weights(weights)
            def act(self, state):
                if np.random.rand() <= self.exploration_rate:
                    return random.randrange(self.action_size)
                act_values = self.brain.predict(state)
                return np.argmax(act_values[0])

            def remember(self, state, action, reward, next_state, done):
                self.memory.append((state, action, reward, next_state, done))
            

In [4]:
def pre_processing(observe):
    processed_observe = np.uint8(
        resize(rgb2gray(observe), (84, 84), mode='constant') * 255)
    return processed_observe

In [5]:
@ray.remote
class CartPole:
    def __init__(self,parameter_servers,gameName):
        self.sample_batch_size = 32
        self.episodes          = 0
        self.env               = gym.make(gameName)
        self.state_size        = (84, 84, 4)
        self.action_size       = self.env.action_space.n
        self.agent             = Agent(self.state_size, self.action_size)
        self.parameter_servers = parameter_servers
        self.memory            = deque(maxlen=400000)
        self.max_steps_per_episode = 10000
        self.total_reward      = 0
    def get_average(self):
      if self.episodes == 0:
        return 0
      return (self.total_reward / self.episodes)
    def run(self):
        epoch = 0
        step = 0
        batch_size = 32
        loss_function = keras.losses.Huber()
        frame_count = 0
        epsilon = 1.0  # Epsilon greedy parameter
        epsilon_min = 0.1  # Minimum epsilon greedy parameter
        epsilon_max = 1.0  # Maximum epsilon greedy parameter
        epsilon_interval = (
            epsilon_max - epsilon_min
        )  # Rate at which to 
        epsilon_random_frames = 50000
        # Number of frames for exploration
        epsilon_greedy_frames = 1000000.0
        high_scores = np.zeros(10)
        last_ten = deque(maxlen=10)
        while True:
            weight  = ray.get(self.parameter_servers.get_weight.remote())
            self.agent.model.set_weights(weight)
            done = False
            dead = False
            episode_reward = 0
            observe = self.env.reset()
            state = pre_processing(observe)
            history = np.stack((state, state, state, state), axis=2)
            history = np.reshape([history], (1, 84, 84, 4))

            while not done:
                #self.env.render()
                frame_count += 1
                step += 1
                # Use epsilon-greedy for exploration
                if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
                    # Take random action
                    action = np.random.choice(self.action_size)
                else:
                    # Predict action Q-values
                    # From environment state
                    state_tensor = tf.convert_to_tensor(state)
                    state_tensor = tf.expand_dims(state_tensor, 0)
                    action_probs = model(state_tensor, training=False)
                    # Take best action
                    action = tf.argmax(action_probs[0]).numpy()
                observe, reward, done, info = self.env.step(action)
                next_state = pre_processing(observe)
                next_state = np.reshape([next_state], (1, 84, 84, 1))
                next_history = np.append(next_state, history[:, :, :, :3], axis=3)
                

                self.memory.append([history,action,reward,next_history,done])
                
                episode_reward += reward
                history = next_history

                if done:
                    last_ten.append(episode_reward)
                    if episode_reward > high_scores[0]: 
                         high_scores[0] = episode_reward
                         # bubble up if necessary 
                         i = 0
                         j = 1
                         while (j < len(high_scores)) and (high_scores[i] > high_scores[j]):
                             temp = high_scores[j]
                             high_scores[j] = high_scores[i] 
                             high_scores[i] = temp 
                             i += 1
                             j += 1
                    globstep = ray.get(self.parameter_servers.getGlobalstep.remote())
                    print("Global step {}".format(globstep))
                    #print("Average score {}".format(np.average(ray.get([actor.get_average.remote() for actor in self.actors]))))
                    print("High scores {}".format(high_scores))
                    print("Last ten {}".format(last_ten))        
                    
                    epoch = epoch + 1
                    self.episodes+=1
                if len(self.memory) >= self.sample_batch_size:
                    mini_batch = random.sample(self.memory, batch_size)
                    states = np.zeros((batch_size, self.state_size[0],self.state_size[1],self.state_size[2]))
                    next_states = np.zeros((batch_size, self.state_size[0],self.state_size[1],self.state_size[2]))
                    actions, rewards, dones = [], [], []
                    target = np.zeros((batch_size))
                    for i in range(batch_size):
                        states[i] = np.float32(mini_batch[i][0] / 255.)
                        actions.append(mini_batch[i][1])
                        rewards.append(mini_batch[i][2])
                        next_states[i] = np.float32(mini_batch[i][3] / 255.)
                        dones.append(mini_batch[i][4])
                        
                    future_rewards = self.agent.target_model.predict(next_states)
                    updated_q_values = rewards + 0.99 * tf.reduce_max(future_rewards, axis=1 )
                    dqn_variable = self.agent.model.trainable_variables
                    masks = tf.one_hot(actions, self.action_size)
                    with tf.GradientTape() as tape:
                       q_values = self.agent.model(states)

                       # Apply the masks to the Q-values to get the Q-value for action taken
                       q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                       # Calculate loss between new Q-value and old Q-value
                       loss = tf.math.reduce_mean(tf.square(q_action - updated_q_values))
                    gradient = tape.gradient(loss, dqn_variable)
                    self.parameter_servers.update_gradient.remote(gradient)
                    globstep = ray.get(self.parameter_servers.getGlobalstep.remote())
                        
                    if globstep > 0 and (globstep % 3000) == 0:
                        print('update target model')
                        w  = ray.get(self.parameter_servers.get_weight.remote())
                        self.agent.target_model.set_weights(self.agent.model.get_weights())

          
              
                
            

In [6]:
import numpy as np
@ray.remote
class ParameterServer(object):
    def __init__(self,gameName):
        # Alternatively, params could be a dictionary mapping keys to arrays.
        self.optimizer = SGD(lr=0.001)
        self.state_size         =  (84, 84, 4)
        self.env               = gym.make(gameName)
        self.action_size       = self.env.action_space.n
        self.model = self.build_model()
        
        self.params = np.zeros(100)
        self.step = 0;
    def get_params(self):
        return self.params

    def update_params(self, grad):
        self.params += grad
    def get_weight(self):
        return self.model.get_weights()
    def build_model(self):
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu',
                         input_shape=self.state_size))
        model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
        model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size))
        model.summary()
        return model
    def getGlobalstep(self):
        return self.step
    def update_gradient(self,grad):
        self.step+=1
        var_list = self.model.trainable_variables
        self.optimizer.apply_gradients(zip(grad, var_list))
        if self.step % 2000 == 0:
              self.model.save("GameModel{:.0f}.h5".format(self.step))
        
      

In [7]:
num_cpus = 3

In [8]:
ray.init(num_cpus=num_cpus,ignore_reinit_error=True)

2021-04-26 23:21:47,201	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


{'metrics_export_port': 51363,
 'node_id': '642b50e731c0997e21b8a831a8efc7793ed3943eec7a37b1821bbe1a',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-04-26_23-21-44_441551_156220/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-04-26_23-21-44_441551_156220/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-04-26_23-21-44_441551_156220',
 'webui_url': '127.0.0.1:8265'}

In [ ]:
GameName = 'SpaceInvaders-v0'
ps = ParameterServer.remote(GameName)

# Start 2 workers.
workers = [CartPole.remote(ps,GameName) for _ in range(num_cpus)]
# Inspect the parameters at regular intervals.

ray.get([worker.run.remote() for worker in workers])

(pid=156381) 2021-04-26 23:21:50.657098: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=156382) 2021-04-26 23:21:50.700457: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", Futur

(pid=156382) Model: "sequential"
(pid=156382) _________________________________________________________________
(pid=156382) Layer (type)                 Output Shape              Param #   
(pid=156382) =================================================================
(pid=156382) conv2d (Conv2D)              (None, 20, 20, 32)        8224      
(pid=156382) _________________________________________________________________
(pid=156382) conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
(pid=156382) _________________________________________________________________
(pid=156382) conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
(pid=156382) _________________________________________________________________
(pid=156382) flatten (Flatten)            (None, 3136)              0         
(pid=156382) _________________________________________________________________
(pid=156382) dense (Dense)                (None, 512)               1606144   
(pid=156382) ______

(pid=156381) 2021-04-26 23:21:59.958151: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=156381) 2021-04-26 23:21:59.964292: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
(pid=156381) 2021-04-26 23:21:59.999766: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(pid=156381) 2021-04-26 23:21:59.999857: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (28c0615dc193): /proc/driver/nvidia/version does not exist
(pid=156381) 2021-04-26 23:22:00.005730: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


(pid=156381) Model: "sequential"
(pid=156381) _________________________________________________________________
(pid=156381) Layer (type)                 Output Shape              Param #   
(pid=156381) =================================================================
(pid=156381) conv2d (Conv2D)              (None, 20, 20, 32)        8224      
(pid=156381) _________________________________________________________________
(pid=156381) conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
(pid=156381) _________________________________________________________________
(pid=156381) conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
(pid=156381) _________________________________________________________________
(pid=156381) flatten (Flatten)            (None, 3136)              0         
(pid=156381) _________________________________________________________________
(pid=156381) dense (Dense)                (None, 512)               1606144   
(pid=156381) ______

(pid=156437) 2021-04-26 23:22:00.791781: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=156437) 2021-04-26 23:22:00.793777: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
(pid=156437) 2021-04-26 23:22:00.822162: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(pid=156437) 2021-04-26 23:22:00.822234: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (28c0615dc193): /proc/driver/nvidia/version does not exist
(pid=156437) 2021-04-26 23:22:00.823423: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=156381) 2021-04-26 23:22:00.940256: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (re

(pid=156437) Model: "sequential"
(pid=156437) _________________________________________________________________
(pid=156437) Layer (type)                 Output Shape              Param #   
(pid=156437) =================================================================
(pid=156437) conv2d (Conv2D)              (None, 20, 20, 32)        8224      
(pid=156437) _________________________________________________________________
(pid=156437) conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
(pid=156437) _________________________________________________________________
(pid=156437) conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
(pid=156437) _________________________________________________________________
(pid=156437) flatten (Flatten)            (None, 3136)              0         
(pid=156437) _________________________________________________________________
(pid=156437) dense (Dense)                (None, 512)               1606144   
(pid=156437) ______

(pid=156436) 2021-04-26 23:22:01.228712: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=156436) 2021-04-26 23:22:01.238198: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
(pid=156436) 2021-04-26 23:22:01.323143: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(pid=156436) 2021-04-26 23:22:01.323204: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (28c0615dc193): /proc/driver/nvidia/version does not exist
(pid=156436) 2021-04-26 23:22:01.324787: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


(pid=156437) Model: "sequential_1"
(pid=156437) _________________________________________________________________
(pid=156437) Layer (type)                 Output Shape              Param #   
(pid=156437) =================================================================
(pid=156437) conv2d_3 (Conv2D)            (None, 20, 20, 32)        8224      
(pid=156437) _________________________________________________________________
(pid=156437) conv2d_4 (Conv2D)            (None, 9, 9, 64)          32832     
(pid=156437) _________________________________________________________________
(pid=156437) conv2d_5 (Conv2D)            (None, 7, 7, 64)          36928     
(pid=156437) _________________________________________________________________
(pid=156437) flatten_1 (Flatten)          (None, 3136)              0         
(pid=156437) _________________________________________________________________
(pid=156437) dense_2 (Dense)              (None, 512)               1606144   
(pid=156437) ____

(pid=156437) 2021-04-26 23:22:01.930741: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
(pid=156437) 2021-04-26 23:22:01.931363: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
(pid=156436) 2021-04-26 23:22:02.335040: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
(pid=156436) 2021-04-26 23:22:02.335606: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
